**Цель проекта**

Построить retrieval-систему, способную находить наиболее релевантные научные статьи из датасета arXiv по текстовому запросу, и продемонстрировать её высокое качество по метрике Mean Reciprocal Rank (MRR@5 > 0.9).

**План работ**

**Этап 1. Исследовательский анализ (EDA).**

- Загрузить данные в Jupyter Notebook. Даны два файла:arxiv-metadata-s.json: содержит метаданные статей (ID, аннотация, название) и test_sample.csv: содержит тестовые запросы и соответствующие им правильные ответы (ID статьи и ее аннотация).
- Проанализировать: кол-во статей в датасете, длину аннотаций и названий, структуру тестовых запросов, возможные проблемы (пропущенные значения, особенности текста).

**Этап 2. Реализация retrieval-системы.**

- Модель эмбеддингов: использовать BAAI/bge-base-en-v1.5 — англоязычнаая.
- Подготовка документов: для каждой статьи из arxiv-metadata-s.json объединить поля title и abstract в единый текст.
- Генерация эмбеддингов корпуса: с помощью model.encode() вычислить эмбеддинги для всех статей.

**Этап 3. Оценка качества и профилирование.**

- Прогнать все тестовые запросы из test_sample.csv.
- Вычислить MRR@5.
- Замерить время:
- Оценка результата MRR@5
- Выводы и рекомендации.

**Мощности, доступные для выполнения проекта**

* видеокарта NVIDIA Tesla T4 (CUDA драйвера установлены);
* операционная система Ubuntu;
* 4 ядра и 16 GB операционной памяти;
* 40 GB SSD;
* предустановлены: командная строка Bash, Python 3.10, pip, unzip.

# Загрузка данных

In [1]:
!wget 'https://www.dropbox.com/scl/fi/1yoqqpw7byz7w7ccx9i2w/requirements.txt?rlkey=bo589ob5nddlgqxzrxfwe2lts&st=1x2hvoa0&dl=1' -O requirements.txt

--2025-12-01 22:18:49--  https://www.dropbox.com/scl/fi/1yoqqpw7byz7w7ccx9i2w/requirements.txt?rlkey=bo589ob5nddlgqxzrxfwe2lts&st=1x2hvoa0&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6028:18::a27d:4712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucca7b0b1abe318257b76135b1da.dl.dropboxusercontent.com/cd/0/inline/C2NnN-_s1bQS6EE8g_2fdCfAtDyPOsmjIdUecS-ieVPTUCEevV-XGJMx-1I8rAWA0U_6Qpvlej38mKrflNMdF1ifGotZl7yLEfAKmFw1NvUr1jKJH0JSqjcQ4UiaBj0ioVX_aB7bW9i0HFKrVzhztCRV/file?dl=1# [following]
--2025-12-01 22:18:50--  https://ucca7b0b1abe318257b76135b1da.dl.dropboxusercontent.com/cd/0/inline/C2NnN-_s1bQS6EE8g_2fdCfAtDyPOsmjIdUecS-ieVPTUCEevV-XGJMx-1I8rAWA0U_6Qpvlej38mKrflNMdF1ifGotZl7yLEfAKmFw1NvUr1jKJH0JSqjcQ4UiaBj0ioVX_aB7bW9i0HFKrVzhztCRV/file?dl=1
Resolving ucca7b0b1abe318257b76135b1da.dl.dropboxusercontent.com (ucca7b0b1abe318257b76135b1da.dl.dropboxuserco

In [ ]:
!pip install -r requirements.txt

In [2]:
import json

In [3]:
!wget "https://www.dropbox.com/scl/fi/xjd1csjaav1nepcltuv4y/arxiv-metadata-s.json?rlkey=gfre8zggjgxdipgbfng74gzd9&st=4a01wlto&dl=1" -O arxiv-metadata-s.json

--2025-12-01 22:18:55--  https://www.dropbox.com/scl/fi/xjd1csjaav1nepcltuv4y/arxiv-metadata-s.json?rlkey=gfre8zggjgxdipgbfng74gzd9&st=4a01wlto&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6028:18::a27d:4712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4839560f7c4a7092b32fe0d1f1.dl.dropboxusercontent.com/cd/0/inline/C2PTSnPw5nO_bIic1EBS_cXkEtt59Pt4ux4pK4wswExp6BRLHH4R0Tvkyp0N6xG64DnaqP7AvPgLGfZze1-k783n9esckXIIMrnppAHkKDSR5Kzq9YYEA60Yubil9IIqcjI_5tQ_Ag0GxGxXcfVp86Iq/file?dl=1# [following]
--2025-12-01 22:18:55--  https://uc4839560f7c4a7092b32fe0d1f1.dl.dropboxusercontent.com/cd/0/inline/C2PTSnPw5nO_bIic1EBS_cXkEtt59Pt4ux4pK4wswExp6BRLHH4R0Tvkyp0N6xG64DnaqP7AvPgLGfZze1-k783n9esckXIIMrnppAHkKDSR5Kzq9YYEA60Yubil9IIqcjI_5tQ_Ag0GxGxXcfVp86Iq/file?dl=1
Resolving uc4839560f7c4a7092b32fe0d1f1.dl.dropboxusercontent.com (uc4839560f7c4a7092b32fe0d1f1.dl.dropboxu

In [4]:
!wget "https://www.dropbox.com/scl/fi/wt0ygvl8qitad8mg4wyv4/test_sample.csv?rlkey=k20omrf2qwcjwnwz6by2b5z90&st=bjdk4932&dl=1" -O test_sample.csv

--2025-12-01 22:19:06--  https://www.dropbox.com/scl/fi/wt0ygvl8qitad8mg4wyv4/test_sample.csv?rlkey=k20omrf2qwcjwnwz6by2b5z90&st=bjdk4932&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6028:18::a27d:4712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc51ef9716d5e62580d36be99c5b.dl.dropboxusercontent.com/cd/0/inline/C2MqlP49lbsyZ2okCyQRXeHPb2zzC6ChAQdxr2wk8pjlOxPNq3DZfB24XUayJAPhjyP2bdvaJZXmRGuRTZUh3LeEUtOsIOt3EqMcuICS7V9OFMJJs5E-S1dnP00yVHYgj2PSlNk2GqJ7fyMqQpq7vXL7/file?dl=1# [following]
--2025-12-01 22:19:07--  https://uc51ef9716d5e62580d36be99c5b.dl.dropboxusercontent.com/cd/0/inline/C2MqlP49lbsyZ2okCyQRXeHPb2zzC6ChAQdxr2wk8pjlOxPNq3DZfB24XUayJAPhjyP2bdvaJZXmRGuRTZUh3LeEUtOsIOt3EqMcuICS7V9OFMJJs5E-S1dnP00yVHYgj2PSlNk2GqJ7fyMqQpq7vXL7/file?dl=1
Resolving uc51ef9716d5e62580d36be99c5b.dl.dropboxusercontent.com (uc51ef9716d5e62580d36be99c5b.dl.dropboxusercon

In [7]:
import pandas as pd

In [8]:
test = pd.read_csv('test_sample.csv')

In [8]:
test[:5]

,id,abstract,query
0,2412.16732,A new platinate was recently discovered when...,What unique composition and decomposition beha...
1,nucl-th/9602019,The production cross sections of various fra...,How does the inclusion of statistical decay af...
2,2501.05500,This survey provides a comprehensive examina...,What are the core components of modern zero-kn...
3,2506.20892,A critical challenge for operating fusion burn...,How does impurity seeding affect the timing an...
4,2208.02031,"In this work, we present the first corpus fo...",What is the primary challenge of the newly dev...


**Все данные успешно загружены! Для простоты восприятия информации (и для удобства последующего анализа) я загружу файл json-формата в виде объектов DataFrame. Приступим к EDA!**

# EDA

In [9]:
df = pd.read_json('arxiv-metadata-s.json', lines=False)

In [10]:
df[:5]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0038,Maxim A. Yurkin,"Maxim A. Yurkin, Alfons G. Hoekstra",The discrete dipole approximation: an overview...,"36 pages, 1 figure; added several corrections ...","J.Quant.Spectrosc.Radiat.Transf. 106, 558-589 ...",10.1016/j.jqsrt.2007.01.034 10.1016/j.jqsrt.20...,None,physics.optics physics.comp-ph,http://creativecommons.org/licenses/by-nc-nd/4.0/,We present a review of the discrete dipole a...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2022-03-30,"[[Yurkin, Maxim A., ], [Hoekstra, Alfons G., ]]"
1,0704.0057,Philipp Werner,Philipp Werner and Andrew J. Millis,High-spin to low-spin and orbital polarization...,Published version,"Phys. Rev. Lett. 99, 126405 (2007)",10.1103/PhysRevLett.99.126405,None,cond-mat.str-el,None,We study the interplay of crystal field spli...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2009-11-13,"[[Werner, Philipp, ], [Millis, Andrew J., ]]"
2,0704.0060,Carlos Bertulani,"C.A. Bertulani, G. Cardella, M. De Napoli, G. ...",Coulomb excitation of unstable nuclei at inter...,"12 pages, 2 figures, accepted for publication ...","Phys.Lett.B650:233-238,2007",10.1016/j.physletb.2007.05.029,None,nucl-th,None,We investigate the Coulomb excitation of low...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-11-26,"[[Bertulani, C. A., ], [Cardella, G., ], [De N..."
3,0704.0070,Yanzhang He,He Yanzhang and Bao Chengguang,Coincidence of the oscillations in the dipole ...,"5 pages, 4 figures, submitted",J. Phys.: Condens. Matter 20 (2008) 055214,None,None,cond-mat.mes-hall,None,The fractional Aharonov-Bohm oscillation (FA...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-19,"[[Yanzhang, He, ], [Chengguang, Bao, ]]"
4,0704.0074,Jawad Y. Abuhlail,"J. Y. Abuhlail, S. K. Nauman",Injective Morita contexts (revisited),None,None,None,None,math.RA,None,This paper is an exposition of the so-called...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2007-08-22,"[[Abuhlail, J. Y., ], [Nauman, S. K., ]]"


In [11]:
df.shape

(98213, 14)

In [ ]:
df.sample(20)

**Что имеем: есть всего около 98 тыс статей, написанный на английском, поэтому в дальнейшем будет логично использовать небольшую модель (в силу небольшого кол-ва данных), обученную на английском корпусе**

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98213 entries, 0 to 98212
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              98213 non-null  object
 1   submitter       97650 non-null  object
 2   authors         98213 non-null  object
 3   title           98213 non-null  object
 4   comments        72275 non-null  object
 5   journal-ref     31672 non-null  object
 6   doi             43744 non-null  object
 7   report-no       6558 non-null   object
 8   categories      98213 non-null  object
 9   license         82351 non-null  object
 10  abstract        98213 non-null  object
 11  versions        98213 non-null  object
 12  update_date     98213 non-null  object
 13  authors_parsed  98213 non-null  object
dtypes: object(14)
memory usage: 10.5+ MB


**В некоторых столбцах отсутствует часть данных: это некритично, так как в дальнейшем мы создадим и сохраним эмбеддинги данных либо из abstract, либо title+abstract (ибо название статьи зачастую содержит краткую информацию о нем, это может быть полезно). В этих двух столбцах все данные имеются, поэтому двигаемся дальше**

**Токенизаторы многих моделей обычно дают похожие результаты, так как они построены на BPE токенизации, поэтому сейчас я использую токенизатор некоторой модели, чтобы выяснить какое кол-во токенов в среднем содержат статьи**

In [10]:
import transformers

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
combined = df['title'] + ' ' + df['abstract']
all_len = combined.apply(lambda x: len(tokenizer.encode(x, truncation=False)))

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors


In [19]:
all_len.describe()

count    98213.000000
mean       226.261045
std         99.451913
min          7.000000
25%        153.000000
50%        216.000000
75%        287.000000
max        945.000000
dtype: float64

In [20]:
all_len.quantile(0.99)

np.float64(497.0)

**Средняя длина статей в токенах - 216, 75-ый квантиль - 287 токенов. Можно сказать, что максимальная длина - 497 токенов (скорее всего, 945 токенов - это выброс).**

**Исходя из длин статей по токенам, можно использовать модель MiniLM-L6-v2 с лимитом токенов 256, + у нее неплохой MTEB-score, но я бы хотела использовать модель, с которой никакие токены не обрежутся (статьи на чанки я делить не будем, в этом нет никакой необходимости, ибо статьи короткие). Поэтому для своей задачи я выбрала модель BGE-base-en-v1.5. У нее в два раза больше параметров, но и макимальная длина последовательности токенов составлят 512, а MTEB-score у нее значительно выше, чем у MiniLM.**

# Реализация retrieval-системы

**Напшем декоратор для профилирования частей кода**

In [13]:
from functools import wraps


def profile(operation_name):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            print(f"{operation_name}")
            start = time.time()
            result = func(self, *args, **kwargs)
            elapsed = time.time() - start

            if hasattr(self, 'profiling_stats'):
                if operation_name not in self.profiling_stats:
                    self.profiling_stats[operation_name] = []
                self.profiling_stats[operation_name].append(elapsed)

            print(f"Завершено за {elapsed:.2f}s")
            return result
        return wrapper
    return decorator

**Создадим класс retrieval системы со всеми методами: загрузка модели, форматирование данных в нужный вид, векторизация и сохранение в векторную БД. Так как создание эмбеддингов занимает немало времени, в классе будут реализованы методы для сохранения/загрузки эмбеддингов**

In [15]:
import time
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import os 

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
class ArXivRetrieval:
    def __init__(self, model_name='BAAI/bge-base-en-v1.5', cache_dir='cache/'):
        self.model_name = model_name
        self.model = None
        self.index = None
        self.metadata = None
        self.embeddings = None 
        self.cache_dir = cache_dir  
        self.profiling_stats = {}
        

    @profile('Загрузка модели')
    def load_model(self):
        self.model = SentenceTransformer(self.model_name, device='cuda')
        return self.model
    

    @profile('Форматирование текстов')
    def prepare_data(self, articles):
        prep_texts = [f"{a['title']} {a['abstract']}" for a in articles]
        return prep_texts
    

    @profile('Векторизация статей')
    def vectorize_data(self, texts, batch_size=128):
        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        return embeddings

    @profile('Создание FAISS индекса')
    def create_index(self, embeddings):
        dimension = embeddings.shape[1]  
        index = faiss.IndexFlatIP(dimension)
        index.add(embeddings)
        return index 
    

    @profile('Создание метаданных')
    def create_metadata(self, articles):
        return pd.DataFrame({
            'id': [a['id'] for a in articles],
            'title': [a['title'] for a in articles],
            'abstract': [a['abstract'] for a in articles]
        })


    def save_cache(self):
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
        
        np.save(os.path.join(self.cache_dir, 'embeddings.npy'), self.embeddings)
        faiss.write_index(self.index, os.path.join(self.cache_dir, 'index.faiss'))
        self.metadata.to_pickle(os.path.join(self.cache_dir, 'metadata.pkl'))
        
        print(f"Кэш сохранён в {self.cache_dir}")
        


    def load_cache(self):
        embeddings_path = os.path.join(self.cache_dir, 'embeddings.npy')
        index_path = os.path.join(self.cache_dir, 'index.faiss')
        metadata_path = os.path.join(self.cache_dir, 'metadata.pkl')
        
        if (os.path.exists(embeddings_path) and 
            os.path.exists(index_path) and 
            os.path.exists(metadata_path)):
            
            self.embeddings = np.load(embeddings_path)
            self.index = faiss.read_index(index_path)
            self.metadata = pd.read_pickle(metadata_path)
            
            print(f"Кэш загружен из {self.cache_dir}")
            return True
        else:
            print(f"Кэш не найден в {self.cache_dir}")
            return False
        

    def build_index(self, articles, batch_size=64):
        if self.load_cache():
            size_mb = self.embeddings.nbytes / (1024**2)
            return {
                'size_mb': size_mb,
                'dimension': self.embeddings.shape[1],
                'num_articles': self.index.ntotal
            }
        
        # Если кэша нет — вычисляем
        texts = self.prepare_data(articles) 
        self.embeddings = self.vectorize_data(texts, batch_size) 
        self.index = self.create_index(self.embeddings) 
        self.metadata = self.create_metadata(articles)
        
        self.save_cache()
        
        size_mb = self.embeddings.nbytes / (1024**2)
        
        return {
            'size_mb': size_mb,
            'dimension': self.embeddings.shape[1],
            'num_articles': self.index.ntotal
        }


    def search(self, query, k=5):
        timings = {}
        
        start = time.time()
        query_emb = self.model.encode(
            [query],
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        timings['query_encoding'] = time.time() - start

        start = time.time()
        distances, indices = self.index.search(query_emb, k)
        timings['faiss_search'] = time.time() - start

        start = time.time()
        results = self.metadata.iloc[indices[0]].copy()
        results['score'] = distances[0]
        timings['metadata_retrieval'] = time.time() - start

        return results, timings
    

    @profile('Вычисление MRR')
    def calculate_mrr(self, test_df, k=5):
        start_total = time.time()  
        
        mrr_scores = []
        all_timings = {
            'query_encoding': [],
            'faiss_search': [],
            'metadata_retrieval': []
        }

        for idx, row in test_df.iterrows():
            query = row['query']
            true_id = row['id']

            results, timings = self.search(query, k=k)

            for key, value in timings.items():
                all_timings[key].append(value)

            found_ids = results['id'].tolist()
            if true_id in found_ids:
                position = found_ids.index(true_id) + 1
                mrr_scores.append(1.0 / position)
            else:
                mrr_scores.append(0.0)

        mrr = np.mean(mrr_scores)
        total_time = time.time() - start_total  

        return {
            'mrr': mrr,
            'total_time': total_time,
            'avg_query_time': total_time / len(test_df),
            'timing_breakdown': all_timings
        }
    

    def get_profiling_report(self):
        print(f"{'='*60}")
        print("Результаты профилирования")
        print(f"{'='*60}")

        report_data = []
        for operation, times in self.profiling_stats.items():
            if isinstance(times, list):
                avg_time = np.mean(times)
                count = len(times)
                total_time = sum(times)
            else:
                avg_time = times
                count = 1
                total_time = times

            report_data.append({
                'Операция': operation,
                'Вызовов': count,
                'Среднее (сек)': f"{avg_time:.4f}",
                'Общее (сек)': f"{total_time:.4f}"
            })

        df = pd.DataFrame(report_data)
        print(df.to_string(index=False))
        return df

In [17]:
model_name = 'BAAI/bge-base-en-v1.5'

retrieval = ArXivRetrieval(model_name)
retrieval.load_model()

Загрузка модели
Завершено за 3.69s


In [19]:
stats = retrieval.build_index(df[['id', 'title', 'abstract']], batch_size=64)

Форматирование текстов
Завершено за 0.07s
Векторизация статей


Batches: 100%|██████████| 1535/1535 [29:26<00:00,  1.15s/it]


Завершено за 1771.76s
Создание FAISS индекса
Завершено за 0.15s
Создание метаданных
Завершено за 0.10s


**Проверим работу системы. Попробуем найти фразу "We investigate the Coulomb excitation..." c id=0704.0060**

In [20]:
results, timings = retrieval.search("We investigate the Coulomb excitation", k=5)
results

,id,title,abstract,score
2,0704.0060,Coulomb excitation of unstable nuclei at inter...,We investigate the Coulomb excitation of low...,0.818065
5767,0912.3189,The Coulomb phase shift revisited,"We investigate the Coulomb phase shift, and ...",0.780614
90897,hep-ph/0103212,Coulomb corrections and multiple e+e- pair pro...,We consider the problem of Coulomb correctio...,0.775762
31684,1711.04342,Coulomb interaction on pion production in Au+A...,Coulomb effects on charged pion transverse m...,0.763313
52268,2107.10436,Classical versus quantum calculation of radiat...,The semiclassical Kepler-Coulomb problem and...,0.759733


In [21]:
timings

{'query_encoding': 0.015112876892089844,
 'faiss_search': 0.02335381507873535,
 'metadata_retrieval': 0.0010514259338378906}

**Итоговая метрика**

In [22]:
mrr_results = retrieval.calculate_mrr(test, k=5)

Вычисление MRR
Завершено за 34.31s


In [25]:
round(mrr_results['mrr'], 2)

np.float64(0.9)

**И результаты профилирования**

In [26]:
report = retrieval.get_profiling_report()

Результаты профилирования
              Операция  Вызовов Среднее (сек) Общее (сек)
       Загрузка модели        1        3.6941      3.6941
Форматирование текстов        2        0.0735      0.1470
   Векторизация статей        1     1771.7573   1771.7573
Создание FAISS индекса        1        0.1501      0.1501
   Создание метаданных        1        0.0986      0.0986
        Вычисление MRR        1       34.3123     34.3123


## Вывод

**В проведённом профилировании видно, что самое узкое место в производительности системы — это этап векторизации статей: он занимает подавляющее большинство времени. Можно значительно ускорить эту часть процесса несколькими способами. Первый подход  — использовать mixed precision, то есть вычисления в формате fp16; обычно это ускоряет обработку и уменьшает использование памяти без потери качества для большинства задач. Второй способ — квантизация модели в формат int8. Это позволило бы уменьшить размер модели в четыре раза и ускорить вычисления в разы при минимальной потере качества (обычно 1-3% метрики MRR)**